In [1]:
import numpy as np
import requests
import json
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt

import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(1, '../')

import apikey
from get_data_API import get_data

# In this notebook
Testing how to use dataframe cleanly with less lines of code and making code more readable

# Get sensor locations from old API

In [2]:
# Set the API endpoint and request all devices
url = 'https://smartcampus.oulu.fi/manage/api/devices/listAll'
old_sensor_data = []

# Send the API request
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    print("200")
else:
    print("Error: API request failed with status code", response.status_code)
    
for line in response.iter_lines(decode_unicode=True):
    old_sensor_data.append(json.loads(line))

200


In [3]:
#First element on the list
print(old_sensor_data[0][0]['deviceId'])
print(old_sensor_data[0][0]['location']['coordinates'])

A81758FFFE046433
[65.05765, 25.46897]


In [4]:
#Dictionary with key and coordinates
sensor_list = {}
for sensor in old_sensor_data[0]:
    
    #Check for Nones in data
    if sensor['location'] == None:
        continue
        
    #without hyphens; ex. A81758FFFE046433
    old_id = sensor['deviceId']
    #with hyphens; ex. A8-17-58-FF-FE-04-64-33
    new_id = '-'.join(old_id[i:i+2] for i in range(0, len(old_id), 2))
    
    #Create dictionary {key new_id : value coordinates}
    sensor_list[new_id] = sensor['location']['coordinates']
    
    
    
sensor_list

{'A8-17-58-FF-FE-04-64-33': [65.05765, 25.46897],
 'A8-17-58-FF-FE-03-0F-6A': [65.05875, 25.46565],
 'A8-17-58-FF-FE-03-10-18': [65.05884, 25.46823],
 'A8-17-58-FF-FE-04-65-00': [65.06081, 25.46679],
 'A8-17-58-FF-FE-03-10-09': [65.05842, 25.46385],
 'A8-17-58-FF-FE-03-10-87': [65.0585, 25.46674],
 'A8-17-58-FF-FE-04-64-F8': [65.05994, 25.46657],
 'A8-17-58-FF-FE-03-0F-8B': [65.05961, 25.466],
 'A8-17-58-FF-FE-03-0F-68': [65.05963, 25.46852],
 'A8-17-58-FF-FE-03-10-0D': [65.05998, 25.46886],
 'A8-17-58-FF-FE-03-0F-93': [65.05965, 25.46385],
 'A8-17-58-FF-FE-04-64-AC': [65.05688, 25.46774],
 'A8-17-58-FF-FE-03-10-90': [65.05807, 25.46889],
 'A8-17-58-FF-FE-03-10-0F': [65.05785, 25.46883],
 'A8-17-58-FF-FE-04-64-B3': [65.05776, 25.46879],
 'A8-17-58-FF-FE-03-10-33': [65.05719, 25.46788],
 'A8-17-58-FF-FE-04-63-38': [65.05967, 25.4634],
 'A8-17-58-FF-FE-03-10-20': [65.05988, 25.46846],
 'A8-17-58-FF-FE-03-0F-D4': [65.06106, 25.46617],
 'A8-17-58-FF-FE-03-0F-B2': [65.06113, 25.46619],
 'A8

# Get sensor data from new API

In [5]:
#smart campus api key
headers = {
    'Content-type': 'application/json', 
    'Authorization': f'{apikey.APIKEY}',
}

In [6]:
# Set the API endpoint and request 15000 datapoints
url = 'https://query-api.rahtiapp.fi/events?limit=15000'

# Send the API request
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the sensor data
    sensor_data = response.iter_lines()
else:
    print("Error: API request failed with status code", response.status_code)

In [7]:
#Create list out of response
resp = []
for line in response.iter_lines(decode_unicode=True):
    resp.append(json.loads(line))

In [8]:
#Singe data point
resp[0][0]

{'time': '2023-04-02T00:00:05.127Z',
 'temperature': 20.7,
 'humidity': 15,
 'light': 14,
 'motion': 2,
 'co2': 537,
 'battery': 3.553,
 'deveui': 'a8-17-58-ff-fe-03-0f-f7',
 'deveui_1': 'a8-17-58-ff-fe-03-0f-f7'}

In [9]:
try:
    sensor_list[resp[0][0]['deveui'].upper()]
    print("Sensor found")
except KeyError:
    print("No locations for sensor")

Sensor found


In [10]:
#Create dataframe
df = pd.DataFrame(columns=['ID','time','temperature','humidity','light','motion','co2', 'coordinates'])

In [11]:
#iterate trough response list
for measurement in resp[0]:
    
    #add row to dataframe
    df.loc[len(df.index)]=[measurement['deveui'],       #id
                           measurement['time'], #timestamp
                           measurement['temperature'],  #temperature
                           measurement['humidity'],     #humidity
                           measurement['light'],        #illuminance
                           measurement['motion'],       #motion
                           measurement['co2'],          #CO2
                           sensor_list[measurement['deveui'].upper()] #coordinates[lat, lng]
                          ]

In [12]:
for measurement in resp[0]:
    print(sensor_list[measurement['deveui'].upper()])

[65.06146, 25.46666]
[65.06129, 25.4691]
[65.05849, 25.46598]
[65.05785, 25.46718]
[65.05806, 25.46922]
[65.0599, 25.46634]
[65.05865, 25.4656]
[65.05882, 25.46564]
[65.05995, 25.46633]
[65.05813404631877, 25.466177165508274]
[65.05916, 25.46565]
[65.05788437215459, 25.46949237585068]
[65.05891, 25.46798]
[65.05786, 25.46883]
[65.05715, 25.46785]
[65.06113, 25.46611]
[65.05769, 25.4676]
[65.0633, 25.46552]
[65.06043, 25.46667]
[65.05895, 25.46782]
[65.05775, 25.46891]
[65.05705, 25.46767]
[65.05791, 25.46887]
[65.06131, 25.46819]
[65.05973, 25.46632]
[65.0606, 25.4667]
[65.05792, 25.46972]
[65.05745, 25.46831]
[65.05782, 25.46833]
[65.05902, 25.46647]
[65.06123, 25.46683]
[65.06124, 25.46738]
[65.06144, 25.46849]
[65.05946, 25.46647]
[65.06142, 25.46646]
[65.05891, 25.46803]
[65.0586, 25.46656]
[65.0585, 25.46674]
[65.05705, 25.46799]
[65.06127, 25.46692]
[65.05725, 25.46777]
[65.05755, 25.46833]
[65.05765, 25.46884]
[65.05844, 25.46672]
[65.05834, 25.46676]
[65.05813404631877, 25.4661

In [13]:
df

,ID,time,temperature,humidity,light,motion,co2,coordinates
0,a8-17-58-ff-fe-03-0f-f7,2023-04-02T00:00:05.127Z,20.7,15,14,2,537,"[65.06146, 25.46666]"
1,a8-17-58-ff-fe-03-0f-5d,2023-04-02T00:00:06.154Z,21.3,12,1,0,516,"[65.06129, 25.4691]"
2,a8-17-58-ff-fe-03-10-91,2023-04-02T00:00:08.866Z,21.3,12,11,0,441,"[65.05849, 25.46598]"
3,a8-17-58-ff-fe-03-0f-ac,2023-04-02T00:00:14.904Z,20.6,15,1,0,539,"[65.05785, 25.46718]"
4,a8-17-58-ff-fe-03-10-8c,2023-04-02T00:00:19.004Z,20.6,16,1,0,516,"[65.05806, 25.46922]"
...,...,...,...,...,...,...,...,...
12788,a8-17-58-ff-fe-03-0f-8a,2023-04-02T11:24:53.344Z,19.9,18,0,1,292,"[65.05692, 25.46841]"
12789,a8-17-58-ff-fe-03-10-74,2023-04-02T11:24:55.769Z,22.2,20,21,0,417,"[65.05973, 25.46816]"
12790,a8-17-58-ff-fe-03-0f-c3,2023-04-02T11:24:57.381Z,22.4,13,59,1,588,"[65.05877, 25.46786]"
12791,a8-17-58-ff-fe-03-0f-ea,2023-04-02T11:24:58.524Z,18.1,15,68,0,525,"[65.05893, 25.46803]"


# Format data to fit HeatMap functions parameters

In [14]:
''' 
HeatMapWithTime()

Parameters
    ----------
    data: list of list of points of the form [lat, lng] or [lat, lng, weight]
        The points you want to plot. The outer list corresponds to the various time
        steps in sequential order. (weight is in (0, 1] range and defaults to 1 if
        not specified for a point)
    index: Index giving the label (or timestamp) of the elements of data. Should have
        the same length as data, or is replaced by a simple count if not specified.
    name : string, default None
        The name of the Layer, as it will appear in LayerControls.
'''

#Min-Max normalize column to get weights between [0, 1] range
column = "motion"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
column = "light"
df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())



In [15]:
motion_data = df[["coordinates", "motion", "time"]]

In [16]:
data = []
timestamps = []

for _, row in motion_data.iterrows():
    data.append([row['coordinates'][0],row['coordinates'][1],row['motion']])
    timestamps.append(row['time'])

data

[[65.06146, 25.46666, 0.010256410256410256],
 [65.06129, 25.4691, 0.0],
 [65.05849, 25.46598, 0.0],
 [65.05785, 25.46718, 0.0],
 [65.05806, 25.46922, 0.0],
 [65.0599, 25.46634, 0.0],
 [65.05865, 25.4656, 0.0],
 [65.05882, 25.46564, 0.0],
 [65.05995, 25.46633, 0.0],
 [65.05813404631877, 25.466177165508274, 0.0],
 [65.05916, 25.46565, 0.0],
 [65.05788437215459, 25.46949237585068, 0.0],
 [65.05891, 25.46798, 0.0],
 [65.05786, 25.46883, 0.0],
 [65.05715, 25.46785, 0.0],
 [65.06113, 25.46611, 0.0],
 [65.05769, 25.4676, 0.0],
 [65.0633, 25.46552, nan],
 [65.06043, 25.46667, 0.005128205128205128],
 [65.05895, 25.46782, 0.4461538461538462],
 [65.05775, 25.46891, 0.0],
 [65.05705, 25.46767, 0.0],
 [65.05791, 25.46887, 0.0],
 [65.06131, 25.46819, 0.005128205128205128],
 [65.05973, 25.46632, 0.4307692307692308],
 [65.0606, 25.4667, 0.0],
 [65.05792, 25.46972, 0.19487179487179487],
 [65.05745, 25.46831, 0.0],
 [65.05782, 25.46833, 0.0],
 [65.05902, 25.46647, 0.0],
 [65.06123, 25.46683, 0.0],
 [65.

# Change time to 24h format

In [17]:
timestamps[-1]

'2023-04-02T11:24:59.975Z'

In [18]:
import datetime
import numpy
from datetime import timedelta

In [19]:
test = datetime.datetime.fromisoformat(timestamps[-1])

In [20]:
test.time()

datetime.time(11, 24, 59, 975000)

In [21]:
datetime.time(7, 45, 0, 0) < test.time()

True

In [22]:
timesOrganized = []
measOrganized = []
measTemp = []
timesTemp = []
slot = None
oldSlot = None
index = 0

def groupData():
    global timesOrganized, measOrganized, measTemp, timesTemp, index, oldSlot, slot
    
    if slot == oldSlot: #Same as previous
        timesTemp.append(time)
        if not numpy.isnan(data[index][2]) and data[index][2] != 0.0:
            measTemp.append(data[index])
    elif oldSlot == None: #First value
        oldSlot = slot
        timesTemp.append(time)
        if not numpy.isnan(data[index][2]) and data[index][2] != 0.0:
            measTemp.append(data[index])
    else: #different as previous
        timesOrganized.append(timesTemp)
        measOrganized.append(measTemp)
        oldSlot = slot
        timesTemp = []
        measTemp = []
        timesTemp.append(time)
        if not numpy.isnan(data[index][2]) and data[index][2] != 0.0:
            measTemp.append(data[index])

for time in timestamps:
    minutes = datetime.datetime.fromisoformat(time).minute
    if minutes >= 0 and minutes < 15 :
        slot = 0
        groupData()
    if minutes >= 15 and minutes < 30 :
        slot = 1
        groupData()
    if minutes >= 30 and minutes < 45 :
        slot = 2
        groupData()
    if minutes >= 45 and minutes < 60 :
        slot = 3
        groupData()
    index = index + 1

In [23]:
i = 0
timeIndexes = []
for i in range(len(timesOrganized)):
    time = datetime.datetime.fromisoformat(timesOrganized[i][-1])
    time = time + timedelta(minutes=1)  
    time = time.replace(second=0, microsecond=0)
    timeIndexes.append(time)

In [24]:
printable = []
for time in timeIndexes:
    printable.append(time.strftime("%m/%d/%Y, %H:%M:%S"))
    
print(printable)
len(printable)


['04/02/2023, 00:15:00', '04/02/2023, 00:30:00', '04/02/2023, 00:45:00', '04/02/2023, 01:00:00', '04/02/2023, 01:15:00', '04/02/2023, 01:30:00', '04/02/2023, 01:45:00', '04/02/2023, 02:00:00', '04/02/2023, 02:15:00', '04/02/2023, 02:30:00', '04/02/2023, 02:45:00', '04/02/2023, 03:00:00', '04/02/2023, 03:15:00', '04/02/2023, 03:30:00', '04/02/2023, 03:45:00', '04/02/2023, 04:00:00', '04/02/2023, 04:15:00', '04/02/2023, 04:30:00', '04/02/2023, 04:45:00', '04/02/2023, 05:00:00', '04/02/2023, 05:15:00', '04/02/2023, 05:30:00', '04/02/2023, 05:45:00', '04/02/2023, 06:00:00', '04/02/2023, 06:15:00', '04/02/2023, 06:30:00', '04/02/2023, 06:45:00', '04/02/2023, 07:00:00', '04/02/2023, 07:15:00', '04/02/2023, 07:30:00', '04/02/2023, 07:45:00', '04/02/2023, 08:00:00', '04/02/2023, 08:15:00', '04/02/2023, 08:30:00', '04/02/2023, 08:45:00', '04/02/2023, 09:00:00', '04/02/2023, 09:15:00', '04/02/2023, 09:30:00', '04/02/2023, 09:45:00', '04/02/2023, 10:00:00', '04/02/2023, 10:15:00', '04/02/2023, 10

45

In [31]:
print(type(measOrganized[0]))
len(measOrganized)

<class 'list'>


45

In [26]:

m = folium.Map([65.06, 25.467], zoom_start=14.5)

#hm = plugins.HeatMapWithTime(measOrganized[1], auto_play=True, max_opacity=0.3)

m.add_child(plugins.HeatMapWithTime(measOrganized, index=printable, auto_play=True, max_opacity=0.3))

#hm.add_to(m)

m